Load dataset

In [2]:
%%capture
!pip install ultralytics
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="Ql1dOvGZp8vqTAnCTntx")
project = rf.workspace("kiyong-kang-qhti3").project("plants-2l8n0")
dataset = project.version(8).download("yolov8")

In [18]:
import os
import shutil
import copy
import yaml
from ultralytics import YOLO
import glob

Segmentation directory

In [4]:
if not os.path.exists('./Plants-seg/train/images'):
    os.makedirs('./Plants-seg/train/images')

if not os.path.exists('./Plants-seg/train/labels'):
    os.makedirs('./Plants-seg/train/labels')

if not os.path.exists('./Plants-seg/valid/images'):
    os.makedirs('./Plants-seg/valid/images')

if not os.path.exists('./Plants-seg/valid/labels'):
    os.makedirs('./Plants-seg/valid/labels')

In [5]:
t_labels = os.listdir('./Plants-8/train/labels')
v_labels = os.listdir('./Plants-8/valid/labels')

train_src = './Plants-8/train'
val_src = './Plants-8/valid'

train_dir = './Plants-seg/train'
val_dir = './Plants-seg/valid'

Prepare for moving tip burn images/labels for segmentation

In [6]:
def prepare_seg_dir(src, labels):
    seg_files = []

    for label_name in labels:
        filename = label_name[:-4]
        label_dir = src + '/labels/' + filename + '.txt'
        with open(label_dir, 'r') as f:
            objs = f.readlines()

        n_obj_tipburn = 0
        objs_tipburn = []
        objs_others = []

        for obj in objs:
            if obj[0] == '0':
                n_obj_tipburn += 1
                objs_tipburn.append(obj)
            else:
                objs_others.append(obj)

        # if len(objs_tipburn) > 0:
        #     if objs_tipburn[-1][-1:] == '\n':
        #         objs_tipburn[-1] = objs_tipburn[-1][:-1]

        # if len(objs_others) > 0:
        #     if objs_others[-1][-1:] == '\n':
        #         objs_others[-1] = objs_others[-1][:-1]

        if n_obj_tipburn == len(objs):
            filename_tipburn = filename + '.tipburn'
            label_dir_tipburn = src + '/labels/' + filename_tipburn + '.txt'
            image_dir = src + '/images/' + filename + '.jpg'
            image_dir_tipburn = src + '/images/' + filename_tipburn + '.jpg'
            seg_files.append(filename_tipburn)
            os.rename(label_dir, label_dir_tipburn)
            os.rename(image_dir, image_dir_tipburn)
        elif n_obj_tipburn < len(objs) and n_obj_tipburn > 0:
            filename_tipburn = filename + '.tipburn'
            label_dir_tipburn = src + '/labels/' + filename_tipburn + '.txt'
            image_dir = src + '/images/' + filename + '.jpg'
            image_dir_tipburn = src + '/images/' + filename_tipburn + '.jpg'
            seg_files.append(filename_tipburn)
            shutil.copy(image_dir, image_dir_tipburn)
            with open(label_dir_tipburn, 'w') as f2:
                f2.writelines(objs_tipburn)
            with open(label_dir, 'w') as f3:
                f3.writelines(objs_others)
        elif n_obj_tipburn == 0:
            continue

    return seg_files

In [7]:
seg_t_files = prepare_seg_dir(train_src, t_labels)
seg_v_files = prepare_seg_dir(val_src, v_labels)

In [8]:
print('Number of train data: ', len(t_labels))
print('Number of train tip burn data: ', len(seg_t_files))
print('Number of valid data: ', len(v_labels))
print('Number of valid valid data: ', len(seg_v_files))

Number of train data:  1517
Number of train tip burn data:  440
Number of valid data:  133
Number of valid valid data:  4


Move tip burn images/labels for segmentation

In [9]:
def move_seg_files(src, dir, seg_files):
    for filename in seg_files:
        shutil.move(src + '/images/' + filename + '.jpg', dir + '/images/' + filename + '.jpg')
        shutil.move(src + '/labels/' + filename + '.txt', dir + '/labels/' + filename + '.txt')

In [10]:
move_seg_files(train_src, train_dir, seg_t_files)
move_seg_files(val_src, val_dir, seg_v_files)

In [11]:
print('Number of train data: ', len(os.listdir(train_src + '/images')), '/', len(os.listdir(train_src + '/labels')))
print('Number of valid data: ', len(os.listdir(val_src + '/images')), '/', len(os.listdir(val_src + '/labels')))
print('Number of tipburn train images/labels: ', len(os.listdir(train_dir + '/images')), '/', len(os.listdir(train_dir + '/labels')))
print('Number of tipburn valid images/labels: ', len(os.listdir(val_dir + '/images')), '/', len(os.listdir(val_dir + '/labels')))

Number of train data:  1499 / 1499
Number of valid data:  133 / 133
Number of tipburn train images/labels:  440 / 440
Number of tipburn valid images/labels:  4 / 4


Modify labels

In [19]:
def modify_labels(dir):
    files = glob.glob(dir)
    for file in files:
        with open(file, 'r') as f:
            labels = f.readlines()
        for i, label in enumerate(labels):
            if label[0] == '1':
                labels[i] = '0' + label[1:]
            elif label[0] == '2':
                labels[i] = '1' + label[1:]

        with open(file, 'w') as f:
            f.writelines(labels)

In [20]:
modify_labels('./Plants-8/train/labels/*.txt')
modify_labels('./Plants-8/valid/labels/*.txt')

Change config yaml for object detection

In [21]:
with open('./Plants-8/data.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    display(config)

{'names': ['Tip_Burn', 'butterhead lettuce', 'romaine lettuce'],
 'nc': 3,
 'roboflow': {'license': 'CC BY 4.0',
  'project': 'plants-2l8n0',
  'url': 'https://universe.roboflow.com/kiyong-kang-qhti3/plants-2l8n0/dataset/8',
  'version': 8,
  'workspace': 'kiyong-kang-qhti3'},
 'test': '../test/images',
 'train': 'Plants-8/train/images',
 'val': 'Plants-8/valid/images'}

In [22]:
config['names'] = ['butterhead lettuce', 'romaine lettuce']
config['nc'] = 2
config['train'] = '/content/Plants-8/train/images'
config['val'] = '/content/Plants-8/valid/images'

In [23]:
with open('./Plants-8/data.yaml', 'w') as file:
    yaml.dump(config, file, default_flow_style=False)

Create config yaml for segmentation and

In [24]:
config_seg = copy.copy(config)
config_seg['names'] = ['Tip_Burn']
config_seg['nc'] = 1
config_seg['train'] = '/content/Plants-seg/train/images'
config_seg['val'] = '/content/Plants-seg/valid/images'

In [25]:
with open('./Plants-seg/data.yaml', 'w') as file:
    yaml.dump(config_seg, file, default_flow_style=False)

Yolov8 - object detection

Load model

In [26]:
model = YOLO('yolov8n.pt')

100%|██████████| 6.23M/6.23M [00:00<00:00, 74.1MB/s]


Train model

In [ ]:
results = model.train(data='/content/Plants-8/data.yaml', epochs=100, imgsz=640)